In [1]:
using Pkg; Pkg.activate("..")
using Plots; pyplot()
using Reproduce
using FileIO
using Statistics
using ProgressMeter
data_home = "../working_results/"

Activating environment at `~/GVFN/Project.toml`


┌ Warning: `getindex(o::PyObject, s::AbstractString)` is deprecated in favor of dot overloading (`getproperty`) so elements should now be accessed as e.g. `o."s"` instead of `o["s"]`.
│   caller = top-level scope at pyplot.jl:24
└ @ Core /home/andrew/.julia/packages/Plots/gylTh/src/backends/pyplot.jl:24


"../working_results/"

In [2]:
function NRMSE(results; phase="Validation", window=1000,skip=10)
    # Ground truth and predictions
    g,p = results["$(phase)GroundTruth"], results["$(phase)Predictions"]

    # drop the extra predictions past the last ground truth
    p = p[1:length(g)]

    # subsample the data for efficiency
    # g, p = g[1:skip:end], p[1:skip:end]

    values = Float64[]
    for i=window+1:skip:length(p)
        ĝ = g[i-window:i]
        P̂ = p[i-window:i]
        push!(values, sqrt(mean((ĝ.-P̂).^2) / mean((ĝ.-mean(ĝ)).^2)))
    end
    return values
end

TrainingNRMSE(results) = NRMSE(results; phase="")
ValidationNRMSE(results) = NRMSE(results; phase="Validation")
TestNRMSE(results) = NRMSE(results; phase="Test")

loadData(itm) = load(joinpath(itm.folder_str,"results.jld2"))["results"]

function get_setting(ic, clean_func)

    diff_dict = diff(ic)
    
    itms = ic.items
    res = Vector{Float64}[]
    for (itm_idx, itm) ∈ enumerate(itms)
        push!(res, clean_func(loadData(itm)))
    end
    
    vals = zeros(length(res[1]),length(res))
    for i=1:length(res)
        vals[:,i] .= res[i]
    end
    
    μ = mean(vals,dims=2)
    σ = std(vals, dims=2, corrected=true)./sqrt(length(itms))
    
    return μ, σ
end

function get_best_setting(ic::ItemCollection, params::Dict)
    sub_ic = ItemCollection(search(ic, params)[3])
    
    sub_diff = diff(sub_ic)
    delete!(sub_diff, "seed")
    
    iter = Iterators.product(values(sub_diff)...)
    best = Inf
    bestIC = nothing
    bestData = nothing
    for arg ∈ iter
        ic′ = ItemCollection(search(sub_ic, Dict(Pair.(keys(sub_diff),arg)))[3])
        μ,_ = get_setting(ic′, ValidationNRMSE)
        
        μ = mean(μ)
        if μ ≤ best
            best = μ
            bestIC = ic′
        end
    end
    @assert best != Inf
    
    return get_setting(ic′, TrainingNRMSE)
end

function get_best_settings(ic::ItemCollection, param::String)
    d = diff(ic)
    params = d[param]
    data = NamedTuple{Vector{Float64},Vector{Float64}}[]
    for p in params
        subIC = search(ic, Dict(param=>p))[3]
        μ, σ = get_best_setting(subIC)
        push!(data, (μ=μ, σ=σ))
    end
    return data.μ, data.σ
end

get_best_settings (generic function with 1 method)

In [3]:
ic = ItemCollection(joinpath(data_home,"mackeyglass_fluxgvfn/data"));

In [4]:
μ,σ = get_best_setting(ic, Dict("tau"=>1))

AssertionError: AssertionError: best != Inf

In [5]:
μ,σ = get_best_setting(ic, Dict("tau"=>16))

AssertionError: AssertionError: best != Inf